# Deploying Machine Learning Models Minikube with RBAC using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)


# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

**2018-06-13** : At present we find the most stable version of minikube across platforms is 0.25.2 as there are issues with 0.26 and 0.27 on some systems. We also find the default VirtualBox driver can be problematic on some systems to we suggest using the [KVM2 driver](https://github.com/kubernetes/minikube/blob/master/docs/drivers.md#kvm2-driver).

Your start command would then look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace "seldon" created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding "kube-system-cluster-admin" created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


Label the node to allow load testing to run on it

In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

deployment "tiller-deploy" successfully rolled out


In [6]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "minikube" labeled


## Start seldon-core

Install the custom resource definition

In [7]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Sep 12 16:35:41 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s


NOTES:
NOTES: TODO




In [8]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon

NAME:   seldon-core
LAST DEPLOYED: Wed Sep 12 16:35:44 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-redis                   1        1        1           0          0s

==> v1/Pod(related)
NAME                                                 READY  STATUS             RESTARTS  AGE
seldon-core-seldon-apiserver-68f5984b49-jmz2w        0/1    ContainerCreating  0         0s
seldon-core-seldon-cluster-manager-7bbc74c894-hjjxq  0/1    Pending            0         0s
seldon-core-redis-575979b79b-frw75                   0/1    Pending            0         0s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  0s

==> v1/ClusterRoleBinding
NAME    AGE


Install prometheus and grafana for analytics

In [9]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

NAME:   seldon-core-analytics
LAST DEPLOYED: Wed Sep 12 16:35:49 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
prometheus  0s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>         0s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-nnw9q      0/1    ContainerCreating  0         0s
alertmanager-deployment-7fbfdfdfb6-9t98j  0/1    ContainerCreating  0         0s
grafana-prom-deployment-7b45fb85d4-sjj99  0/1    ContainerCreating  0         0s
prometheus-node-exporter-wcxb8            0/1    ContainerCreating  0         0s
prometheus-deployment-cbfd78cc7-wcr52     0/1    Pending            0         0s

==> v1/Servi

Check all services are running before proceeding.

In [10]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver

Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward the seldon api-server REST and GRPC ports**:

REST:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

GRPC:
```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:5000
```

Install gRPC modules for the prediction protos.

In [11]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [11]:
!kubectl create -f resources/model_invalid1.json -n seldon

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha2", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"clusterName":"", "labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "namespace":"seldon", "creationTimestamp":"2018-09-08T17:28:16Z", "uid":"92581429-b38c-11e8-8d19-f8fff9063696", "selfLink":""}, "spec":map[string]interface {}{"annotations":map[string]interface {}{"deployment_version":"v1", "project_name":"FX Market Prediction"}, "name":"test-deployment", "oauth_key":1234, "oauth_secret":"oauth-secret", "predictors":[]interface {}{map[string]interface {}{"annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpecs":[]interface {}{map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[string]interface {}{"image":"seldonio/mean_classifier:0.6", "imagePullPolicy":22, "name":"mean-classifie

## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [12]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [13]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [14]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicas:1 replicasAvailable:1 name:test-deployment-example-svc-orch] map[replicasAvailable:1 name:test-deployment-example-classifier-0 replicas:1]]]

## Get predictions

In [15]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8002"
API_GATEWAY_GRPC="localhost:8003"

#### REST Request

In [16]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"d16a212f-c861-45de-bf5f-c634cf602d5c","token_type":"bearer","expires_in":43199,"scope":"read write"}
{
  "meta": {
    "puid": "urvtkn3uh34bguifguqiihd3vd",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07598352200745992]
    }
  }
}


#### gRPC Request

In [17]:
 grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"d16a212f-c861-45de-bf5f-c634cf602d5c","token_type":"bearer","expires_in":43197,"scope":"read write"}
meta {
  puid: "6s3l23rl05qasv9cdkhkq12hrh"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07028283389333956
  }
}



## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [18]:
!pygmentize resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

In [19]:
!kubectl apply -f resources/model_with_canary.json -n seldon

seldondeployment "seldon-model" configured


Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [21]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1] map[name:test-deployment-canary-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-canary-mean-classifier-0 replicas:1 replicasAvailable:1]]]

#### REST Request

In [22]:
rest_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST)

{"access_token":"d16a212f-c861-45de-bf5f-c634cf602d5c","token_type":"bearer","expires_in":43117,"scope":"read write"}
{
  "meta": {
    "puid": "35fc3sdl0jj0m8s6lco0kq9bt5",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07515448234459429]
    }
  }
}


#### gRPC request

In [23]:
grpc_request_api_gateway("oauth-key","oauth-secret",API_GATEWAY_REST,API_GATEWAY_GRPC)

{"access_token":"d16a212f-c861-45de-bf5f-c634cf602d5c","token_type":"bearer","expires_in":43115,"scope":"read write"}
meta {
  puid: "t7b9vir0mcvfoa9llvi6jbvs8h"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08294586370978486
  }
}



## Load test

Start a load test which will post REST requests at 10 requests per second.

In [24]:
!helm install seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://seldon-core-seldon-apiserver:8080 \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --namespace seldon \
    --repo https://storage.googleapis.com/seldon-charts

NAME:   loadtest
LAST DEPLOYED: Sat Sep  8 18:32:36 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP     EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.111.138.47  <none>       5557:31357/TCP,5558:31829/TCP,8089:31564/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-pc6mc   0/1    ContainerCreating  0         0s
locust-master-1-8g9jr  0/1    ContainerCreating  0         0s




You should port-foward the grafana dashboard

```bash
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then view an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core-analytics above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [ ]:
!helm delete loadtest --purge

In [ ]:
!kubectl delete -f resources/model_with_canary.json -n seldon

In [ ]:
!helm delete seldon-core-analytics --purge

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge